In [65]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [75]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [76]:
soup = BeautifulSoup(page.content,'lxml')

<h1> Scrape the table of Postal codes from webpage and assign column names </h1>

In [68]:
match = soup.title.text
match = soup.find('div',class_='mw-parser-output')
table = soup.find_all('table')[0] 
raw_df = pd.read_html(str(table))
raw_df= raw_df[0]
raw_df.columns=['Postcode','Borough','Neighbourhood']

 <h1> Filter the cells that have an assigned borough. Ignore cells with a borough that is Not assigned </h1>

In [69]:
has_value = raw_df['Borough']!='Not assigned'
df_with_borough = raw_df[has_value]
df_with_borough = df_with_borough.drop(df_with_borough.index[0])

<h1> Group 'Neighbourhood' data based on Postcode and Borough </h1>

In [70]:
df_grouped = df_with_borough.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda Neighbourhood: ','.join(Neighbourhood)).reset_index()
df_grouped.columns=['Postcode','Borough','Neighbourhood']

<h1> Replace 'Not assigned' Neighbourhood with the value in Borough </h1>

In [71]:
df_grouped.Neighbourhood.replace('Not assigned',df_grouped.Borough, inplace=True)

<h1> Print the number of rows of your dataframe </h1>

In [74]:
df_grouped.shape

(103, 3)